In [ ]:
!pip install super-image lpips -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.0/91.0 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.8/53.8 kB 4.5 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
!unzip /content/drive/MyDrive/skin_cancer_dataset/dataset.zip -d dataset

Archive:  /content/drive/MyDrive/skin_cancer_dataset/dataset.zip
   creating: dataset/dataset/
  inflating: dataset/__MACOSX/._dataset  
  inflating: dataset/dataset/.DS_Store  
  inflating: dataset/__MACOSX/dataset/._.DS_Store  
   creating: dataset/dataset/test/
  inflating: dataset/__MACOSX/dataset/._test  
   creating: dataset/dataset/train/
  inflating: dataset/__MACOSX/dataset/._train  
   creating: dataset/dataset/validation/
  inflating: dataset/__MACOSX/dataset/._validation  
  inflating: dataset/dataset/test/.DS_Store  
  inflating: dataset/__MACOSX/dataset/test/._.DS_Store  
   creating: dataset/dataset/test/BCC/
  inflating: dataset/__MACOSX/dataset/test/._BCC  
   creating: dataset/dataset/test/SCC/
  inflating: dataset/__MACOSX/dataset/test/._SCC  
   creating: dataset/dataset/test/MEL/
  inflating: dataset/__MACOSX/dataset/test/._MEL  
   creating: dataset/dataset/train/BCC/
  inflating: dataset/__MACOSX/dataset/train/._BCC  
   creating: dataset/dataset/train/SCC/
  inf

In [ ]:
!unzip /content/drive/MyDrive/skin_cancer_dataset/super-resolution.zip -d super-resolution

Archive:  /content/drive/MyDrive/skin_cancer_dataset/super-resolution.zip
   creating: super-resolution/super-resolution/
   creating: super-resolution/super-resolution/edsr-base/
   creating: super-resolution/super-resolution/edsr-base/test/
   creating: super-resolution/super-resolution/edsr-base/test/BCC/
  inflating: super-resolution/super-resolution/edsr-base/test/BCC/scaled_2x_PAT_82_125_525.png  
  inflating: super-resolution/super-resolution/edsr-base/test/BCC/scaled_2x_PAT_727_1373_776.png  
  inflating: super-resolution/super-resolution/edsr-base/test/BCC/scaled_2x_PAT_830_1564_740.png  
  inflating: super-resolution/super-resolution/edsr-base/test/BCC/scaled_2x_PAT_837_1582_714.png  
  inflating: super-resolution/super-resolution/edsr-base/test/BCC/scaled_2x_PAT_237_363_344.png  
  inflating: super-resolution/super-resolution/edsr-base/test/BCC/scaled_2x_PAT_96_150_743.png  
  inflating: super-resolution/super-resolution/edsr-base/test/BCC/scaled_2x_PAT_522_986_800.png  
  i

In [ ]:
import os
from PIL import Image
from super_image import EdsrModel, ImageLoader

In [ ]:
import torch

# Check if GPU is available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("device", device)

# Main execution
input_dir = 'dataset/dataset/test/MEL/'
output_dir = 'super-resolution/edsr-base'

os.makedirs(output_dir, exist_ok=True)

# Load model and move it to the appropriate device
model = EdsrModel.from_pretrained('eugenesiow/edsr-base', scale=2).to(device)

def process_images(input_dir, output_dir, model):
    model.eval()  # Set the model to evaluation mode
    for root, dirs, files in os.walk(input_dir):
        for filename in files:
            if filename.lower().endswith(('.jpg', '.jpeg', '.png')):

                input_path = os.path.join(root, filename)

                rel_path = os.path.relpath(root, input_dir)
                output_subdir = os.path.join(output_dir, rel_path)
                os.makedirs(output_subdir, exist_ok=True)

                output_path = os.path.join(output_subdir, f'scaled_2x_{filename}')
                compare_path = os.path.join(output_subdir, f'scaled_2x_compare_{filename}')

                try:
                    with Image.open(input_path) as image:
                        inputs = ImageLoader.load_image(image)
                        # Move inputs to the same device as the model
                        inputs = inputs.to(device)
                        preds = model(inputs)
                        # Save the upscaled image
                        ImageLoader.save_image(preds, output_path)
                        # Save the comparison image
                        # ImageLoader.save_compare(inputs, preds, compare_path)

                    print(f"Processed {input_path}")
                except Exception as e:
                    print(f"Error processing {input_path}: {e}")

process_images(input_dir, output_dir, model)


device cuda


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:131: FutureWarning: 'cached_download' (from 'huggingface_hub.file_download') is deprecated and will be removed from version '0.26'. Use `hf_hub_download` instead.
  warnings.warn(warning_message, FutureWarning)
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:680: FutureWarning: 'cached_download' is the legacy way to download files from the HF hub, please consider upgrading to 'hf_hub_download'
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public m

config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

pytorch_model_2x.pt:   0%|          | 0.00/5.51M [00:00<?, ?B/s]

https://huggingface.co/eugenesiow/edsr-base/resolve/main/pytorch_model_2x.pt


/usr/local/lib/python3.10/dist-packages/super_image/modeling_utils.py:247: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(resolved_archive_file, map_l

Processed dataset/dataset/test/MEL/PAT_710_1330_243.png
Error processing dataset/dataset/test/MEL/PAT_936_1772_886.png: CUDA out of memory. Tried to allocate 1.72 GiB. GPU 0 has a total capacity of 39.56 GiB of which 974.81 MiB is free. Process 11574 has 38.60 GiB memory in use. Of the allocated memory 37.99 GiB is allocated by PyTorch, and 132.91 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)
Processed dataset/dataset/test/MEL/PAT_109_868_723.png
Processed dataset/dataset/test/MEL/PAT_495_941_26.png
Processed dataset/dataset/test/MEL/PAT_966_1825_584.png
Error processing dataset/dataset/test/MEL/PAT_754_1429_380.png: CUDA out of memory. Tried to allocate 1.04 GiB. GPU 0 has a total capacity of 39.56 GiB of which 692.81 MiB is free. Process 11574 has

In [ ]:
!nvidia-smi

Wed Oct  2 08:26:58 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   54C    P0              27W /  70W |  14765MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
import os
import torch
from super_image import EdsrModel, ImageLoader
from PIL import Image
import gc

def process_image(input_path, output_path, model, device):
    try:
        with Image.open(input_path) as image:
            # Resize image if it's too large
            max_size = 1024  # Adjust this value based on your GPU memory
            if max(image.size) > max_size:
                image.thumbnail((max_size, max_size), Image.LANCZOS)

            inputs = ImageLoader.load_image(image).to(device)

            # Process in chunks if the image is still too large
            chunk_size = 512  # Adjust this value based on your GPU memory
            height, width = inputs.shape[2], inputs.shape[3]

            output = torch.zeros((1, 3, height*2, width*2), device='cpu')

            for y in range(0, height, chunk_size):
                for x in range(0, width, chunk_size):
                    chunk = inputs[:, :, y:y+chunk_size, x:x+chunk_size]
                    with torch.cuda.amp.autocast():  # Use mixed precision
                        preds = model(chunk)
                    output[:, :, y*2:(y+chunk_size)*2, x*2:(x+chunk_size)*2] = preds.cpu()

                    # Clear cache after each chunk
                    torch.cuda.empty_cache()

            # Save the upscaled image
            ImageLoader.save_image(output, output_path)

        print(f"Processed {input_path}")
    except Exception as e:
        print(f"Error processing {input_path}: {e}")

def process_images(input_dir, output_dir, model, device):
    for root, dirs, files in os.walk(input_dir):
        for filename in files:
            if filename.lower().endswith(('.jpg', '.jpeg', '.png')):
                input_path = os.path.join(root, filename)
                rel_path = os.path.relpath(root, input_dir)
                output_subdir = os.path.join(output_dir, rel_path)
                os.makedirs(output_subdir, exist_ok=True)
                output_path = os.path.join(output_subdir, f'scaled_2x_{filename}')

                process_image(input_path, output_path, model, device)

                # Clear cache after each image
                torch.cuda.empty_cache()
                gc.collect()

# Main execution
#if __name__ == "__main__":
input_dir = 'dataset/dataset/validation/SCC/'
output_dir = 'super-resolution/edsr-base/validation/SCC/'
os.makedirs(output_dir, exist_ok=True)

# Set PyTorch to use TF32 (faster on A100)
torch.backends.cuda.matmul.allow_tf32 = True
torch.backends.cudnn.allow_tf32 = True

# Check if CUDA is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Load the model and move it to the GPU
model = EdsrModel.from_pretrained('eugenesiow/edsr-base', scale=2)
model = model.to(device)
#model.eval()  # Set the model to evaluation mode

# Enable gradient scaler for mixed precision
scaler = torch.cuda.amp.GradScaler()

process_images(input_dir, output_dir, model, device)

Using device: cuda
https://huggingface.co/eugenesiow/edsr-base/resolve/main/pytorch_model_2x.pt


<ipython-input-17-40e7d99cec88>:76: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler()
<ipython-input-17-40e7d99cec88>:26: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():  # Use mixed precision


Processed dataset/dataset/validation/SCC/PAT_713_1336_159.png
Processed dataset/dataset/validation/SCC/PAT_834_1572_430.png
Processed dataset/dataset/validation/SCC/PAT_663_1254_732.png
Processed dataset/dataset/validation/SCC/PAT_482_924_134.png
Processed dataset/dataset/validation/SCC/PAT_113_172_610.png
Processed dataset/dataset/validation/SCC/PAT_771_1488_922.png
Processed dataset/dataset/validation/SCC/PAT_51_78_959.png
Processed dataset/dataset/validation/SCC/PAT_48_74_402.png
Processed dataset/dataset/validation/SCC/PAT_131_197_974.png
Processed dataset/dataset/validation/SCC/PAT_990_1860_636.png
Processed dataset/dataset/validation/SCC/PAT_490_933_352.png
Processed dataset/dataset/validation/SCC/PAT_369_755_733.png
Processed dataset/dataset/validation/SCC/PAT_809_1527_902.png
Processed dataset/dataset/validation/SCC/PAT_874_1672_287.png
Processed dataset/dataset/validation/SCC/PAT_192_296_824.png
Processed dataset/dataset/validation/SCC/PAT_325_689_528.png
Processed dataset/dat

In [ ]:
!cp -r '/content/super-resolution' '/content/drive/MyDrive/skin_cancer_dataset/'

In [ ]:
!zip -r super-resolution.zip super-resolution

  adding: super-resolution/ (stored 0%)
  adding: super-resolution/edsr-base/ (stored 0%)
  adding: super-resolution/edsr-base/test/ (stored 0%)
  adding: super-resolution/edsr-base/test/BCC/ (stored 0%)
  adding: super-resolution/edsr-base/test/BCC/scaled_2x_PAT_82_125_525.png (deflated 1%)
  adding: super-resolution/edsr-base/test/BCC/scaled_2x_PAT_727_1373_776.png (deflated 4%)
  adding: super-resolution/edsr-base/test/BCC/scaled_2x_PAT_830_1564_740.png (deflated 1%)
  adding: super-resolution/edsr-base/test/BCC/scaled_2x_PAT_837_1582_714.png (deflated 5%)
  adding: super-resolution/edsr-base/test/BCC/scaled_2x_PAT_237_363_344.png (deflated 0%)
  adding: super-resolution/edsr-base/test/BCC/scaled_2x_PAT_96_150_743.png (deflated 3%)
  adding: super-resolution/edsr-base/test/BCC/scaled_2x_PAT_522_986_800.png (deflated 3%)
  adding: super-resolution/edsr-base/test/BCC/scaled_2x_PAT_88_133_64.png (deflated 1%)
  adding: super-resolution/edsr-base/test/BCC/scaled_2x_PAT_522_986_305.png (

In [ ]:
!cp -r '/content/super-resolution.zip' '/content/drive/MyDrive/skin_cancer_dataset/'

### Evaluation

In [ ]:
import numpy as np
from PIL import Image
from skimage.metrics import peak_signal_noise_ratio as psnr
from skimage.metrics import structural_similarity as ssim
from scipy.stats import pearsonr
import lpips
import torch
from super_image import EdsrModel, ImageLoader

def calculate_psnr(img1, img2):
    return psnr(img1, img2)

def calculate_ssim(img1, img2, win_size=None):
    # If win_size is None, calculate the largest possible odd window size
    if win_size is None:
        win_size = min(img1.shape[0], img1.shape[1])
        win_size = win_size if win_size % 2 != 0 else win_size - 1

    # Ensure win_size is at least 3
    win_size = max(3, win_size)

    return ssim(img1, img2, win_size=win_size, channel_axis=2)

def calculate_pearson_correlation(img1, img2):
    return pearsonr(img1.flatten(), img2.flatten())[0]

def calculate_lpips(img1, img2, lpips_model, device):
    # Convert images to tensors
    tensor1 = torch.from_numpy(img1).permute(2, 0, 1).unsqueeze(0).float() / 255.0
    tensor2 = torch.from_numpy(img2).permute(2, 0, 1).unsqueeze(0).float() / 255.0

    # Move tensors to the specified device
    tensor1 = tensor1.to(device)
    tensor2 = tensor2.to(device)

    with torch.no_grad():
        lpips_score = lpips_model(tensor1, tensor2).item()
    return lpips_score

def evaluate_image_quality(original_path, sr_path):
    # Load images
    original_img = np.array(Image.open(original_path).convert('RGB'))
    sr_img = np.array(Image.open(sr_path).convert('RGB'))

    # Ensure images are the same size (resize original to match SR if necessary)
    if original_img.shape != sr_img.shape:
        original_img = np.array(Image.open(original_path).convert('RGB').resize(sr_img.shape[:2][::-1], Image.BICUBIC))

    # Calculate metrics
    psnr_value = calculate_psnr(original_img, sr_img)
    ssim_value = calculate_ssim(original_img, sr_img)
    pearson_value = calculate_pearson_correlation(original_img, sr_img)

    # LPIPS
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    lpips_model = lpips.LPIPS(net='alex').to(device)
    lpips_value = calculate_lpips(original_img, sr_img, lpips_model, device)

    return {
        'PSNR': psnr_value,
        'SSIM': ssim_value,
        'Pearson Correlation': pearson_value,
        'LPIPS': lpips_value
    }

def super_resolve_image(input_path, output_path):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = EdsrModel.from_pretrained('eugenesiow/edsr-base', scale=2).to(device)
    model.eval()

    with Image.open(input_path) as img:
        inputs = ImageLoader.load_image(img).to(device)
        with torch.no_grad():
            preds = model(inputs)
        ImageLoader.save_image(preds, output_path)

In [ ]:
if __name__ == "__main__":
    original_image_path = '/content/dataset/dataset/test/BCC/PAT_115_177_622.png'
    sr_output_path = '/content/super-resolution/super-resolution/edsr-base/test/BCC/scaled_2x_PAT_115_177_622.png'

    # Generate super-resolved image
    super_resolve_image(original_image_path, sr_output_path)

    # Evaluate image quality
    metrics = evaluate_image_quality(original_image_path, sr_output_path)

    print("Image Quality Metrics:")
    for metric, value in metrics.items():
        print(f"{metric}: {value}")

https://huggingface.co/eugenesiow/edsr-base/resolve/main/pytorch_model_2x.pt
Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]
Loading model from: /usr/local/lib/python3.10/dist-packages/lpips/weights/v0.1/alex.pth
Image Quality Metrics:
PSNR: 42.180797521655705
SSIM: 0.9976885304746629
Pearson Correlation: 0.9994883696295191
LPIPS: 0.0272224061191082


In [ ]:
import os
import numpy as np
from PIL import Image
from skimage.metrics import peak_signal_noise_ratio as psnr
from skimage.metrics import structural_similarity as ssim
from scipy.stats import pearsonr
import lpips
import torch
import csv
from tqdm import tqdm

def calculate_psnr(img1, img2):
    return psnr(img1, img2)

def calculate_ssim(img1, img2, win_size=None):
    if win_size is None:
        win_size = min(img1.shape[0], img1.shape[1])
        win_size = win_size if win_size % 2 != 0 else win_size - 1
    win_size = max(3, win_size)
    return ssim(img1, img2, win_size=win_size, channel_axis=2)

def calculate_pearson_correlation(img1, img2):
    return pearsonr(img1.flatten(), img2.flatten())[0]

def calculate_lpips(img1, img2, lpips_model, device):
    tensor1 = torch.from_numpy(img1).permute(2, 0, 1).unsqueeze(0).float() / 255.0
    tensor2 = torch.from_numpy(img2).permute(2, 0, 1).unsqueeze(0).float() / 255.0
    tensor1 = tensor1.to(device)
    tensor2 = tensor2.to(device)
    with torch.no_grad():
        lpips_score = lpips_model(tensor1, tensor2).item()
    return lpips_score

def evaluate_image_quality(original_path, sr_path, lpips_model, device):
    original_img = np.array(Image.open(original_path).convert('RGB'))
    sr_img = np.array(Image.open(sr_path).convert('RGB'))

    if original_img.shape != sr_img.shape:
        original_img = np.array(Image.open(original_path).convert('RGB').resize(sr_img.shape[:2][::-1], Image.BICUBIC))

    psnr_value = calculate_psnr(original_img, sr_img)
    ssim_value = calculate_ssim(original_img, sr_img)
    pearson_value = calculate_pearson_correlation(original_img, sr_img)
    lpips_value = calculate_lpips(original_img, sr_img, lpips_model, device)

    return {
        'PSNR': psnr_value,
        'SSIM': ssim_value,
        'Pearson Correlation': pearson_value,
        'LPIPS': lpips_value
    }

def process_folders(original_base_dir, sr_base_dir, output_csv):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    lpips_model = lpips.LPIPS(net='alex').to(device)

    subfolders = ['test', 'train', 'validation']
    categories = ['BCC', 'MEL', 'SCC']

    with open(output_csv, 'w', newline='') as csvfile:
        fieldnames = ['Set', 'Category', 'Image', 'PSNR', 'SSIM', 'Pearson Correlation', 'LPIPS']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()

        for subfolder in subfolders:
            for category in categories:
                original_folder = os.path.join(original_base_dir, 'dataset', subfolder, category)
                sr_folder = os.path.join(sr_base_dir, 'edsr-base', subfolder, category)

                if not os.path.exists(original_folder) or not os.path.exists(sr_folder):
                    print("original_folder", original_folder)
                    print("sr_folder", sr_folder)
                    print(f"Skipping {subfolder}/{category} - folder not found")
                    continue

                original_images = [f for f in os.listdir(original_folder) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]

                for image_name in tqdm(original_images, desc=f"Processing {subfolder}/{category}"):
                    original_path = os.path.join(original_folder, image_name)
                    sr_path = os.path.join(sr_folder, f"scaled_2x_{image_name}")

                    if not os.path.exists(sr_path):
                        print(f"Warning: Super-resolved image not found for {image_name}")
                        continue

                    try:
                        metrics = evaluate_image_quality(original_path, sr_path, lpips_model, device)
                        writer.writerow({
                            'Set': subfolder,
                            'Category': category,
                            'Image': image_name,
                            **metrics
                        })
                    except Exception as e:
                        print(f"Error processing {image_name}: {e}")

In [ ]:
if __name__ == "__main__":
    original_base_dir = '/content/dataset/'
    sr_base_dir = '/content/super-resolution/super-resolution'
    output_csv = 'image_quality_metrics.csv'

    process_folders(original_base_dir, sr_base_dir, output_csv)
    print(f"Evaluation complete. Results saved to {output_csv}")

Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]
Loading model from: /usr/local/lib/python3.10/dist-packages/lpips/weights/v0.1/alex.pth


Processing validation/SCC: 100%|██████████| 82/82 [02:11<00:00,  1.61s/it]

Evaluation complete. Results saved to image_quality_metrics.csv


In [ ]:
import pandas as pd
import numpy as np

def extended_metrics_analysis(csv_file):
    df = pd.read_csv(csv_file)

    # Calculate overall averages and standard deviations
    overall_stats = df[['PSNR', 'SSIM', 'Pearson Correlation', 'LPIPS']].agg(['mean', 'std'])
    print("Overall Statistics:")
    print(overall_stats)
    print("\n")

    # Find best and worst images based on PSNR
    best_psnr = df.loc[df['PSNR'].idxmax()]
    worst_psnr = df.loc[df['PSNR'].idxmin()]
    print("Best PSNR Image:")
    print(best_psnr)
    print("\nWorst PSNR Image:")
    print(worst_psnr)
    print("\n")

    # Calculate averages and standard deviations by Set and Category
    set_category_stats = df.groupby(['Set', 'Category'])[['PSNR', 'SSIM', 'Pearson Correlation', 'LPIPS']].agg(['mean', 'std'])
    print("Statistics by Set and Category:")
    print(set_category_stats)
    print("\n")

    # Optional: Save the results to CSV
    overall_stats.to_csv('overall_stats.csv')
    set_category_stats.to_csv('set_category_stats.csv')

if __name__ == "__main__":
    csv_file = 'image_quality_metrics.csv'  # Replace with your CSV file name if different
    extended_metrics_analysis(csv_file)

Overall Statistics:
           PSNR      SSIM  Pearson Correlation     LPIPS
mean  46.581166  0.998008             0.998961  0.030275
std    4.117950  0.002903             0.001716  0.020424


Best PSNR Image:
Set                                    test
Category                                BCC
Image                  PAT_551_1047_567.png
PSNR                              55.291775
SSIM                               0.999956
Pearson Correlation                0.999963
LPIPS                              0.008629
Name: 96, dtype: object

Worst PSNR Image:
Set                                   train
Category                                BCC
Image                  PAT_747_1408_742.png
PSNR                              29.829264
SSIM                               0.972788
Pearson Correlation                0.982249
LPIPS                               0.07881
Name: 458, dtype: object


Statistics by Set and Category:
                          PSNR                SSIM            \
        

In [ ]:
import os
import numpy as np
from PIL import Image
from skimage.metrics import peak_signal_noise_ratio as psnr
from skimage.metrics import structural_similarity as ssim
from scipy.stats import pearsonr
import lpips
import torch
import csv
from tqdm import tqdm
import pandas as pd

def calculate_psnr(img1, img2):
    return psnr(img1, img2)

def calculate_ssim(img1, img2):
    return ssim(img1, img2, channel_axis=2)

def calculate_pearson_correlation(img1, img2):
    return pearsonr(img1.flatten(), img2.flatten())[0]

def calculate_lpips(img1, img2, lpips_model, device):
    tensor1 = torch.from_numpy(img1).permute(2, 0, 1).unsqueeze(0).float() / 255.0
    tensor2 = torch.from_numpy(img2).permute(2, 0, 1).unsqueeze(0).float() / 255.0
    tensor1 = tensor1.to(device)
    tensor2 = tensor2.to(device)
    with torch.no_grad():
        lpips_score = lpips_model(tensor1, tensor2).item()
    return lpips_score

def bicubic_upscale(image_path, scale_factor=2):
    with Image.open(image_path) as img:
        w, h = img.size
        return img.resize((w * scale_factor, h * scale_factor), Image.BICUBIC)

def process_images(dataset_dir, output_dir, scale_factor=2):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    lpips_model = lpips.LPIPS(net='alex').to(device)

    results = []

    for set_folder in ['test', 'train', 'validation']:
        set_path = os.path.join(dataset_dir, set_folder)
        if not os.path.exists(set_path):
            print(f"Warning: {set_folder} folder not found in {dataset_dir}")
            continue

        for category in ['BCC', 'MEL', 'SCC']:
            category_path = os.path.join(set_path, category)
            if not os.path.exists(category_path):
                print(f"Warning: {category} folder not found in {set_path}")
                continue

            output_category_path = os.path.join(output_dir, set_folder, category)
            os.makedirs(output_category_path, exist_ok=True)

            for file in tqdm(os.listdir(category_path), desc=f"Processing {set_folder}/{category}"):
                if file.lower().endswith(('.png', '.jpg', '.jpeg')):
                    original_path = os.path.join(category_path, file)
                    output_path = os.path.join(output_category_path, f'bicubic_2x_{file}')

                    # Upscale image
                    upscaled_img = bicubic_upscale(original_path, scale_factor)
                    upscaled_img.save(output_path)

                    # Calculate metrics
                    original_img = np.array(Image.open(original_path).convert('RGB'))
                    upscaled_arr = np.array(upscaled_img.convert('RGB'))

                    # Resize original to match upscaled size for comparison
                    original_resized = np.array(Image.fromarray(original_img).resize(upscaled_img.size, Image.BICUBIC))

                    psnr_value = calculate_psnr(original_resized, upscaled_arr)
                    ssim_value = calculate_ssim(original_resized, upscaled_arr)
                    pearson_value = calculate_pearson_correlation(original_resized, upscaled_arr)
                    lpips_value = calculate_lpips(original_resized, upscaled_arr, lpips_model, device)

                    results.append({
                        'Set': set_folder,
                        'Category': category,
                        'File': file,
                        'PSNR': psnr_value,
                        'SSIM': ssim_value,
                        'Pearson Correlation': pearson_value,
                        'LPIPS': lpips_value
                    })

    return results

def save_results(results, output_csv):
    with open(output_csv, 'w', newline='') as csvfile:
        fieldnames = ['Set', 'Category', 'File', 'PSNR', 'SSIM', 'Pearson Correlation', 'LPIPS']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()
        for row in results:
            writer.writerow(row)

In [ ]:
if __name__ == "__main__":
    dataset_dir = '/content/dataset/dataset'
    output_dir = 'bicubic/upscaled/images'
    output_csv = 'bicubic_upscaling_metrics.csv'

    results = process_images(dataset_dir, output_dir)
    save_results(results, output_csv)

    print(f"Bicubic upscaling complete. Results saved to {output_csv}")

    # Calculate and print average metrics
    df = pd.read_csv(output_csv)
    average_metrics = df.groupby(['Set', 'Category'])[['PSNR', 'SSIM', 'Pearson Correlation', 'LPIPS']].mean()
    print("\nAverage Metrics for Bicubic Upscaling by Set and Category:")
    print(average_metrics)

    overall_average = df[['PSNR', 'SSIM', 'Pearson Correlation', 'LPIPS']].mean()
    print("\nOverall Average Metrics for Bicubic Upscaling:")
    print(overall_average)

Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
/usr/local/lib/python3.10/dist-packages/lpips/lpips.py:107: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted

Loading model from: /usr/local/lib/python3.10/dist-packages/lpips/weights/v0.1/alex.pth


Processing test/BCC:   0%|          | 0/127 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/skimage/metrics/simple_metrics.py:168: RuntimeWarning: divide by zero encountered in scalar divide
  return 10 * np.log10((data_range**2) / err)
Processing validation/SCC: 100%|██████████| 82/82 [04:43<00:00,  3.46s/it]

Bicubic upscaling complete. Results saved to bicubic_upscaling_metrics.csv

Average Metrics for Bicubic Upscaling by Set and Category:
                     PSNR  SSIM  Pearson Correlation  LPIPS
Set        Category                                        
test       BCC        inf   1.0                  1.0    0.0
           MEL        inf   1.0                  1.0    0.0
           SCC        inf   1.0                  1.0    0.0
train      BCC        inf   1.0                  1.0    0.0
           MEL        inf   1.0                  1.0    0.0
           SCC        inf   1.0                  1.0    0.0
validation BCC        inf   1.0                  1.0    0.0
           MEL        inf   1.0                  1.0    0.0
           SCC        inf   1.0                  1.0    0.0

Overall Average Metrics for Bicubic Upscaling:
PSNR                   inf
SSIM                   1.0
Pearson Correlation    1.0
LPIPS                  0.0
dtype: float64


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import os
from PIL import Image
import numpy as np

def load_and_preprocess_data(bicubic_csv, sr_csv):
    bicubic_df = pd.read_csv(bicubic_csv)
    sr_df = pd.read_csv(sr_csv)

    # Ensure both dataframes have the same images
    common_images = set(bicubic_df['File']) & set(sr_df['File'])
    bicubic_df = bicubic_df[bicubic_df['File'].isin(common_images)]
    sr_df = sr_df[sr_df['File'].isin(common_images)]

    return bicubic_df, sr_df

def compare_metrics(bicubic_df, sr_df):
    metrics = ['PSNR', 'SSIM', 'Pearson Correlation', 'LPIPS']
    sets = ['test', 'train', 'validation']
    categories = ['BCC', 'MEL', 'SCC']

    for set_name in sets:
        for category in categories:
            print(f"\nComparison of Average Metrics for {set_name.capitalize()} Set, {category} Category:")
            subset_bicubic = bicubic_df[(bicubic_df['Set'] == set_name) & (bicubic_df['Category'] == category)]
            subset_sr = sr_df[(sr_df['Set'] == set_name) & (sr_df['Category'] == category)]

            for metric in metrics:
                bicubic_mean = subset_bicubic[metric].mean()
                sr_mean = subset_sr[metric].mean()
                improvement = (sr_mean - bicubic_mean) / bicubic_mean * 100

                print(f"{metric}:")
                print(f"  Bicubic: {bicubic_mean:.4f}")
                print(f"  Super-Resolution: {sr_mean:.4f}")
                print(f"  Improvement: {improvement:.2f}%")

                # Perform t-test
                t_stat, p_value = stats.ttest_rel(subset_bicubic[metric], subset_sr[metric])
                print(f"  T-test p-value: {p_value:.4f}")
                print()

def visualize_metrics(bicubic_df, sr_df):
    metrics = ['PSNR', 'SSIM', 'Pearson Correlation', 'LPIPS']

    for set_name in ['test', 'train', 'validation']:
        for category in ['BCC', 'MEL', 'SCC']:
            fig, axes = plt.subplots(2, 2, figsize=(15, 15))
            fig.suptitle(f'Metric Comparison for {set_name.capitalize()} Set, {category} Category')
            axes = axes.flatten()

            subset_bicubic = bicubic_df[(bicubic_df['Set'] == set_name) & (bicubic_df['Category'] == category)]
            subset_sr = sr_df[(sr_df['Set'] == set_name) & (sr_df['Category'] == category)]

            for i, metric in enumerate(metrics):
                data = pd.DataFrame({
                    'Bicubic': subset_bicubic[metric],
                    'Super-Resolution': subset_sr[metric]
                })
                sns.boxplot(data=pd.melt(data), x='variable', y='value', ax=axes[i])
                axes[i].set_title(metric)
                axes[i].set_xlabel('')

            plt.tight_layout()
            plt.savefig(f'metric_comparison_{set_name}_{category}.png')
            plt.close()

def visualize_image_samples(bicubic_dir, sr_dir, original_dir, num_samples=2):
    sets = ['test', 'train', 'validation']
    categories = ['BCC', 'MEL', 'SCC']

    for set_name in sets:
        for category in categories:
            bicubic_folder = os.path.join(bicubic_dir, set_name, category)
            sr_folder = os.path.join(sr_dir, set_name, category)
            original_folder = os.path.join(original_dir, set_name, category)

            bicubic_images = [f for f in os.listdir(bicubic_folder) if f.endswith(('.png', '.jpg', '.jpeg'))]
            samples = np.random.choice(bicubic_images, min(num_samples, len(bicubic_images)), replace=False)

            #fig, axes = plt.subplots(len(samples), 3, figsize=(15, 5*len(samples)))
            #fig.suptitle(f'Image Samples for {set_name.capitalize()} Set, {category} Category')
            fig, axes = plt.subplots(len(samples), 2, figsize=(15, 5*len(samples)))
            fig.suptitle(f'{category} Category')

            for i, sample in enumerate(samples):
                original_img = Image.open(os.path.join(original_folder, sample.replace('bicubic_2x_', '')))
                bicubic_img = Image.open(os.path.join(bicubic_folder, sample))
                sr_img = Image.open(os.path.join(sr_folder, sample.replace('bicubic_2x_', 'scaled_2x_')))

                #axes[i, 0].imshow(original_img)
                #axes[i, 0].set_title('Original')
                axes[i, 0].imshow(bicubic_img)
                axes[i, 0].set_title('Original')
                axes[i, 1].imshow(sr_img)
                axes[i, 1].set_title('Enhanced')

                for ax in axes[i]:
                    ax.axis('off')

            plt.tight_layout()
            plt.savefig(f'image_samples_comparison_{set_name}_{category}.png')
            plt.close()

if __name__ == "__main__":
    bicubic_csv = 'bicubic_upscaling_metrics.csv'
    sr_csv = 'image_quality_metrics.csv'
    bicubic_dir = 'bicubic/upscaled/images'
    sr_dir = '/content/super-resolution/super-resolution/edsr-base'
    original_dir = '/content/dataset/dataset'

    #bicubic_df, sr_df = load_and_preprocess_data(bicubic_csv, sr_csv)
    #compare_metrics(bicubic_df, sr_df)
    #visualize_metrics(bicubic_df, sr_df)
    visualize_image_samples(bicubic_dir, sr_dir, original_dir)

    print("Comparison complete. Check the generated PNG files for visualizations.")

en(samples) 2
en(samples) 2
en(samples) 2
en(samples) 2
en(samples) 2
en(samples) 2
en(samples) 2
en(samples) 2
en(samples) 2
Comparison complete. Check the generated PNG files for visualizations.
